*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Sentiment analysis using BERT

In [1]:
# This code is obtained from the Text classification notebook
import sys
sys.path.append("../../")
import os
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.dataset.bert_sentiment import download_and_load_datasets
from utils_nlp.eval.classification import eval_classification
from utils_nlp.models.bert.sequence_classification import BERTSequenceClassifier
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.common.timer import Timer
import torch
import torch.nn as nn
import numpy as np

In this notebook, we follow along with the [text-classification notebook](tc_mnli_bert.ipynb) to fine-tune BERT to perform sentiment analysis on the IMDB dataset [IMDB Large movie reviews](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) hosted by Stanford. The following data pre-processing is obtained from Google Research's example notebook for fine-tuning BERT.

In [2]:
BERT_CACHE_DIR = "../../../temp"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE = 32
NUM_GPUS = 2
NUM_EPOCHS = 2
TRAIN_SIZE = 0.6
LABEL_COL = "polarity"
TEXT_COL = "sentence"

## Data processing

In [3]:
# Get the dataset and save it into pandas Dataframe
train, test = download_and_load_datasets()

=====> Begin downloading
=====> Done downloading
=====> Finish extracting
=============> Complete downloading
**** Dataset path: C:\Users\ducl\Documents\GitHub\nlp-2\scenarios\text_classification\data\aclImdb
===> Directory: C:\Users\ducl\Documents\GitHub\nlp-2\scenarios\text_classification\data\aclImdb\train


KeyboardInterrupt: 

Keep in mind that this dataset contains movie reviews (as a whole) instead of individual sentences. Therefore, in the case of BERT, we should note that a single input in this case is a paragraph with potentially multiple sentences, compare to the original version where each input are a single sentence.

In [ ]:
train.head(5)

In [ ]:
train.info()

The dataset is divided equally into 2 group of polarity, with 1 is positive review and 0 is negative review. The sentiment value are more detailed about the actual reaction rate, but we will experiemtn with it later.

In [ ]:
train.polarity.value_counts()

In [ ]:
# Take a sample of the data
df_train = train.sample(frac=1)
df_test = test.sample(frac=1)

We encode the class labels to make sure that we know which is which

In [ ]:
# encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(labels_train))

In [ ]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

## Tokenize and Preprocess

Before training, we need to transform the text data into a format that BERT understands. This process involves two steps. First, we instantiate a BERT tokenizer with a given language and then tokenize the text of the training and testing sets.

In [ ]:
tokenizer = Tokenizer(LANGUAGE, to_lower=TO_LOWER, cache_dir=BERT_CACHE_DIR)

tokens_train = tokenizer.tokenize(list(df_train[TEXT_COL]))
tokens_test = tokenizer.tokenize(list(df_test[TEXT_COL]))

Second, we perform the following preprocessing steps in the cell below:
- Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
- Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
- Pad or truncate the token lists to the specified max length
- Return mask lists that indicate paddings' positions
- Return token type id lists that indicate which sentence the tokens belong to (not needed for one-sequence classification)

*See the original [implementation](https://github.com/google-research/bert/blob/master/run_classifier.py) for more information on BERT's input format.*

In [ ]:
tokens_train, mask_train, _ = tokenizer.preprocess_classification_tokens(
    tokens_train, MAX_LEN
)
tokens_test, mask_test, _ = tokenizer.preprocess_classification_tokens(
    tokens_test, MAX_LEN
)

## Create Model
Next, we create a sequence classifier that loads a pre-trained BERT model, given the language and number of labels.

In [ ]:
classifier = BERTSequenceClassifier(
    language=LANGUAGE, num_labels=num_labels, cache_dir=BERT_CACHE_DIR
)

## Train
We train the classifier using the training examples. This involves fine-tuning Hugging Face's PyTorch implementation of a pre-trained BERT transformer with a linear classifier layer attached to perform sequence classification tasks. The arguments for fitting the classifier are:
- token_ids: list of token indices
- input_mask: mask lists that indicate paddings' position
- labels: list of training labels
- num_gpus: number of GPUs. If none specified, all available GPUs will be used
- num_epochs: number of training epochs (default 1)
- batch_size: training batch size (default 32)
- verbose: displays training progress and loss values

In [ ]:
with Timer() as t:
    classifier.fit(
        token_ids=tokens_train,
        input_mask=mask_train,
        labels=labels_train,    
        num_gpus=NUM_GPUS,        
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,    
        verbose=True,
    )    
print("[Training time: {:.3f} hrs]".format(t.interval / 3600))

## Score
We score the test set using the trained classifier:

In [ ]:
preds = classifier.predict(
    token_ids=tokens_test, input_mask=mask_test, num_gpus=NUM_GPUS, batch_size=BATCH_SIZE
)

## Evaluate Results
Finally, we compute the accuracy, precision, recall, and F1 metrics of the evaluation on the test set.

In [ ]:
print(classification_report(labels_test, preds, target_names=["negative", "positive"]))